In [ ]:
import os
import cv2 as cv2
from matplotlib import pyplot as plt

In [ ]:
img_path = 'images/at_1.png'

In [ ]:
img = cv2.imread(img_path)
plt.imshow(img)
plt.show()

In [ ]:
mask, masked_img = pcv.threshold.custom_range(img=img, lower_thresh=[0,0,158], upper_thresh=[255,255,255], channel='LAB')
